In [1]:
import pandas as pd
from sklearn.calibration import LabelEncoder
import numpy as np
# 1. Load the data
df = pd.read_csv('./megablunders.csv')

In [2]:
# look at class imbalances

import matplotlib.pyplot as plt

# get the count for each label
print(df["error"].value_counts())

df_filtered = df[df["error"] != "AGREEERROR"]

# Verify the class has been removed
print("\nClass distribution after dropping:")
print(df_filtered["error"].value_counts())

# Update your DataFrame
df = df_filtered


print(df["error"].value_counts())

error
DM       22
PR       22
PAR      22
ROS      22
FRAG     20
NONE     20
MM       20
CASE     19
AGREE    18
Name: count, dtype: int64

Class distribution after dropping:
error
DM       22
PR       22
PAR      22
ROS      22
FRAG     20
NONE     20
MM       20
CASE     19
AGREE    18
Name: count, dtype: int64
error
DM       22
PR       22
PAR      22
ROS      22
FRAG     20
NONE     20
MM       20
CASE     19
AGREE    18
Name: count, dtype: int64


# Expanding the Dataset

In [3]:
# 

In [4]:
# use lable encoder to encode the error
label_encoder = LabelEncoder()
df['error'] = label_encoder.fit_transform(df['error'])

# Model

In [5]:
#  vectorize the text 
from sentence_transformers import SentenceTransformer

st = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

df = df.reset_index(drop=True)

/Users/felixstuart/Documents/School/Clubs/Programming Club/Potential Projects/ML Megablunders/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# swap text vectorization to POS embedding
from flair.data import Sentence
from flair.models import SequenceTagger
from sklearn.preprocessing import OneHotEncoder


# load tagger



label_encoder = LabelEncoder()
label_encoder.fit([
    "", "ADD", "AFX", "CC", "CD", "DT", "EX", "FW", "HYPH", "IN", "JJ", "JJR", "JJS",
    "LS", "MD", "NFP", "NN", "NNP", "NNPS", "NNS", "PDT", "POS", "PRP", "PRP$",
    "RB", "RBR", "RBS", "RP", "SYM", "TO", "UH", "VB", "VBD", "VBG", "VBN", "VBP",
    "VBZ", "WDT", "WP", "WP$", "WRB", "XX",
    ".", ",", ":", "``", "''", "-LRB-", "-RRB-", "#", "$"
])


tagger = SequenceTagger.load("flair/pos-english")

one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
one_hot_encoder.fit(label_encoder.transform(label_encoder.classes_).reshape(-1, 1))
    
# make example sentence
sentence = Sentence("I love Berlin.")


def pos_encode(original_sentence):
    sentence = Sentence(original_sentence)
    tagger.predict(sentence)
    
    tags = [entity._value for entity in sentence.get_labels('pos')]
    indices = label_encoder.transform(tags)
    
    return indices
    
embeddings = df["original_sentence"].apply(lambda x: pos_encode(x))

2025-05-18 21:19:03,853 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD


In [7]:
df.head()

,original_sentence,error
0,By dropping a game to the pathetic Tampa Bay D...,2
1,"Although, if history is any indication, the te...",3
2,"The Sox almost never go down uneventfully, whi...",7
3,Because of the accumulated bad karma that hang...,0
4,The team not only has squandered huge leads bu...,6


In [8]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(embeddings, df['error'], test_size=0.1, random_state=42, stratify=df["error"])

In [9]:
print(embeddings)

0      [17, 41, 13, 24, 17, 13, 18, 25, 25, 25, 26, 4...
1      [17, 4, 17, 24, 44, 13, 24, 4, 13, 24, 22, 32,...
2      [13, 26, 32, 32, 43, 32, 32, 4, 45, 44, 13, 34...
3      [17, 17, 13, 42, 18, 24, 45, 44, 17, 13, 24, 4...
4      [13, 24, 32, 32, 44, 42, 18, 27, 11, 32, 30, 4...
                             ...                        
180    [24, 44, 18, 4, 11, 13, 24, 18, 17, 18, 27, 44...
181    [17, 13, 27, 17, 41, 17, 24, 4, 25, 44, 42, 32...
182    [37, 39, 17, 24, 4, 25, 44, 42, 32, 4, 42, 31,...
183    [13, 24, 40, 17, 13, 18, 24, 4, 45, 40, 32, 42...
184    [13, 24, 32, 40, 17, 13, 18, 24, 4, 45, 40, 32...
Name: original_sentence, Length: 185, dtype: object


In [10]:
# pad the sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences

embeddings = pad_sequences(embeddings, 24)




In [11]:
import torch 

embeddings = torch.tensor(embeddings)

In [12]:
import torch.nn as nn

position_embeddings = nn.Embedding(50,128)

embeddings = position_embeddings(embeddings)

embeddings.shape

IndexError: index out of range in self

In [ ]:
# try xgboost
from sklearn.utils import compute_class_weight
import xgboost as xgb
from sklearn.metrics import accuracy_score

# Convert class weights to sample weights
# Create a dictionary mapping class labels to weights

unique_classes = np.unique(y_train)

weights = compute_class_weight(
    class_weight="balanced",
    classes=unique_classes,
    y=y_train
)

class_weight_dict = dict(zip(unique_classes, weights))

# Map each sample to its appropriate weight
sample_weights = np.array([class_weight_dict[cls] for cls in y_train])

# Fit the XGBoost model with sample weights
xboost = xgb.XGBClassifier(n_estimators=5000,
                           learning_rate=0.1,
                           random_state=42,
                        #    reg_alpha=0.7,
                        #    reg_lambda=1,
                        #    max_depth=3,
                           scale_pos_weight=sample_weights,
                           )


xboost.fit(X_train, y_train, sample_weight=sample_weights)

# Make predictions
y_pred = xboost.predict(X_test)

# Evaluate
accuracy = accuracy_score(y_test, y_pred)
print(f"XGBoost Accuracy: {accuracy*100:.2f}%")

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, the experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:original_sentence: object